In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('bhw_ais_ship_2024.csv.gz')
df.head()

,mmsi,imo,name,type,length,width,yday
0,413000000,NaN,NaN,70.0,0.0,0.0,1
1,412000000,NaN,0,NaN,NaN,NaN,1
2,414352330,NaN,NaN,31.0,0.0,0.0,1
3,413355510,0.0,HUA TAI 21,70.0,149.0,21.0,1
4,897898896,NaN,XIPU NAVIGATION,NaN,NaN,NaN,1


In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df=df.sort_values(by=['mmsi']).reset_index(drop=True)

In [5]:
mask=(df['yday']>=92)&(df['yday']<=121)
df=df[mask].reset_index(drop=True)

In [6]:
def pick_type(series):
    s=pd.to_numeric(series, errors='coerce').dropna()
    if s.empty:
        return np.nan
    #首选合法值
    valid_small=s[(s>0)&(s<=100)]
    if not valid_small.empty:
        counts=valid_small.value_counts()
        modes=counts[counts==counts.max()].index
        return int(modes.min())
    positive=s[s>100]
    #大于100的赋值哨兵值-1（实际上异常值）
    if not positive.empty:
        return int(-1)
    #NaN和0赋值哨兵值-1
    return int(-1)

def pick_length(series):
    s=pd.to_numeric(series,errors='coerce')
    positive=s[s>0]
    if not positive.empty:
        return positive.min()
    return int(-1)

unique_ship_info=df.groupby('mmsi').agg({
    'type':lambda x: pick_type(x),
    'length':lambda x: pick_length(x),
}).reset_index()

In [7]:
df=pd.read_csv('bhw_total_ais_2024.csv.gz')

In [8]:
df['time']=pd.to_datetime(df['time'],errors='coerce')

In [11]:
#筛选出2024年4月的数据
start_time_second_unix=pd.Timestamp('2024-04-01 00:00:00',tz='UTC')
end_time_second_unix=pd.Timestamp('2024-05-01 00:00:00',tz='UTC')
df=df[(df['time']>=start_time_second_unix)&(df['time']<end_time_second_unix)]

In [12]:
df=df.merge(unique_ship_info,on='mmsi',how='left')

In [13]:
df.to_csv('cleaned_ais.csv',index=False)